In [1]:
!pip3 install torch torchvision torchaudio

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms

In [4]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [5]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transform
)

100%|██████████| 26421880/26421880 [00:13<00:00, 2023724.00it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 170411.68it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:08<00:00, 552637.00it/s] 


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5694165.87it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [6]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transform
)

In [7]:
batch_size = 32

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X,y in train_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([32, 1, 28, 28])
Shape of y: torch.Size([32]) torch.int64


In [8]:
# Create the Mode

class FashionMNIST(nn.Module):

  def __init__(self):
    super(FashionMNIST, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [9]:
model = FashionMNIST()
print(model)

FashionMNIST(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
torch.cuda.is_available()

True

In [11]:
device = "cuda"

In [12]:
# move to GPU
model.to(device)

FashionMNIST(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [13]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-3)

In [14]:
def train(dataloader, model, loss_function, optimizer):
  size = len(dataloader.dataset) # complete length of dataset
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device) # move the dataset to GPU
    pred = model(X) # Forward pass - it will track the operation
    loss = loss_function(pred, y) # calculate the value of loss
    optimizer.zero_grad() # before the update of gradients we will set to zero values of gradients

    #Backpropagation
    loss.backward() # find the gradients of each parameter
    optimizer.step() # update the parameters --> w = w - eta*gradients
    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [15]:
def test(dataloader, model, loss_function):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_function(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

In [16]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader, model, loss_function, optimizer)
  test(test_dataloader, model, loss_function)
print("Done!")

Epoch 1
-------------------------------
loss: 2.307186  [    0/60000]
loss: 2.283629  [ 3200/60000]
loss: 2.272037  [ 6400/60000]
loss: 2.239395  [ 9600/60000]
loss: 2.224751  [12800/60000]
loss: 2.218415  [16000/60000]
loss: 2.199730  [19200/60000]
loss: 2.180042  [22400/60000]
loss: 2.177967  [25600/60000]
loss: 2.111504  [28800/60000]
loss: 2.062240  [32000/60000]
loss: 2.039189  [35200/60000]
loss: 2.125679  [38400/60000]
loss: 2.018993  [41600/60000]
loss: 1.969000  [44800/60000]
loss: 1.959030  [48000/60000]
loss: 2.011970  [51200/60000]
loss: 1.915120  [54400/60000]
loss: 1.823732  [57600/60000]
Epoch 2
-------------------------------
loss: 1.823276  [    0/60000]
loss: 1.829294  [ 3200/60000]
loss: 1.765087  [ 6400/60000]
loss: 1.706996  [ 9600/60000]
loss: 1.642568  [12800/60000]
loss: 1.710619  [16000/60000]
loss: 1.511212  [19200/60000]
loss: 1.512646  [22400/60000]
loss: 1.531285  [25600/60000]
loss: 1.481420  [28800/60000]
loss: 1.343347  [32000/60000]
loss: 1.288608  [352

In [17]:
torch.save(model.state_dict(), "model.pth")
print("Saved the pytorch model to model.pth")

Saved the pytorch model to model.pth


In [21]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.to("cpu")
model.eval()
X, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  pred = model(X)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
